字典允许我们通过键的方式来创建嵌套结构进而存储数据，而键并不存储实际的数据，只是一个索引，索引的对象存储真正的数据。因此，字典的数据结构其实是很多K/V对的组合，举个最简单的例子：

In [2]:
# 最简单的字典
d = {"name": 'stanley', "age": 18, "love": 'python'}

# 取值的方式也很简单：
d["name"]

'stanley'

什么是扁平化字典
了解什么是字典，扁平化字典会容易了解很多了，直观讲就是键值对的嵌套而已。我们来看实际例子：

In [3]:
ansible_fact = {
    "sda": {
        "holders": [], 
        "host": "SATA controller: Intel Corporation 82801HR/HO/HH (ICH8R/DO/DH) 6 port SATA Controller [AHCI mode] (rev 02)", 
        "model": "CentOS Linux-0 S", 
        "partitions": {
            "sda1": {
                "holders": [], 
                "sectors": "1024000", 
                "sectorsize": 512, 
                "size": "500.00 MB", 
                "start": "2048", 
                "uuid": "3aec5112-2a1c-4ce0-91de-e9514ba357b8"
            }, 
            "sda2": {
                "holders": [
                    "VolGroup-lv_root", 
                    "VolGroup-lv_swap", 
                    "VolGroup-lv_home"
                ], 
                "sectors": "133191680", 
                "sectorsize": 512, 
                "size": "63.51 GB", 
                "start": "1026048", 
                "uuid": 'null'
            }
        }
    }
} 

如上是ansible的setup获取的信息输出片段，如我们期望取 ansible_fact => sda => partitions => sda2 => size 的值该怎么办呢？

In [4]:
ansible_fact['sda']['partitions']['sda2']['size']

'63.51 GB'

这样取值确实很麻烦，如果我们想取值方便将其转化为扁平化数组是不错的方式（只是通过该需求来演示扁平化数组，该案例的需求在实际企业中也可能是伪需求），ok我们看下扁平化后的数组是什么样子的：

In [5]:
{'sda/holders': [],
 'sda/host': 'SATA controller: Intel Corporation 82801HR/HO/HH (ICH8R/DO/DH) 6 port SATA Controller [AHCI mode] (rev 02)',
 'sda/model': 'CentOS Linux-0 S',
 'sda/partitions/sda1/holders': [],
 'sda/partitions/sda1/sectors': '1024000',
 'sda/partitions/sda1/sectorsize': 512,
 'sda/partitions/sda1/size': '500.00 MB',
 'sda/partitions/sda1/start': '2048',
 'sda/partitions/sda1/uuid': '3aec5112-2a1c-4ce0-91de-e9514ba357b8',
 'sda/partitions/sda2/holders': ['VolGroup-lv_root',
                                                                                                      'VolGroup-lv_swap',
                                 'VolGroup-lv_home'],
 'sda/partitions/sda2/sectors': '133191680',
 'sda/partitions/sda2/sectorsize': 512,
 'sda/partitions/sda2/size': '63.51 GB',
 'sda/partitions/sda2/start': '1026048',
 'sda/partitions/sda2/uuid': 'null'}

{'sda/holders': [],
 'sda/host': 'SATA controller: Intel Corporation 82801HR/HO/HH (ICH8R/DO/DH) 6 port SATA Controller [AHCI mode] (rev 02)',
 'sda/model': 'CentOS Linux-0 S',
 'sda/partitions/sda1/holders': [],
 'sda/partitions/sda1/sectors': '1024000',
 'sda/partitions/sda1/sectorsize': 512,
 'sda/partitions/sda1/size': '500.00 MB',
 'sda/partitions/sda1/start': '2048',
 'sda/partitions/sda1/uuid': '3aec5112-2a1c-4ce0-91de-e9514ba357b8',
 'sda/partitions/sda2/holders': ['VolGroup-lv_root',
  'VolGroup-lv_swap',
  'VolGroup-lv_home'],
 'sda/partitions/sda2/sectors': '133191680',
 'sda/partitions/sda2/sectorsize': 512,
 'sda/partitions/sda2/size': '63.51 GB',
 'sda/partitions/sda2/start': '1026048',
 'sda/partitions/sda2/uuid': 'null'}

k值中的/可以是任意你期望的连接符，这里以/做示例，这样对比下来大家对扁平化字典有清晰的了解了吧。我们来看下具体的实现思路

## 实现思路
相信聪明的朋友们都看出来了，无非是将原来的字典通过一定的规则转化为新的字典而已，那这里的大致思路及涉及技术点如下：
	1. 获取旧字典的K/V对（迭代器）
	2. 对K/V做循环，直至不再嵌套字典 （循环、判断）
	3. 使用/拼接嵌套循环的K1、K2…Kn （format）
	4. 对K1/K2/../Kn赋值并生成新的字典（字典）
    
## 涉及技术点

### 字典
最简单的字典
```python
d = {"name": stanley, "age": 18, "love": 'python'}
```
### 字典更新:
通过索引更新字典
```python
d['base'] = 'china'
```
迭代
函数
def fun(args):
pass
fun(arg)
函数嵌套
def fun(args):
def fun2(arg2):
pass
fun2(arg2)
fun(arg)
format格式化输出
```python
'i am {},i love {}'.format('stanley','python')
```

In [6]:
def flatten(d):
    def _flatten(src,dst,pattern=''):
        for k,v in src.items():
            key = k if pattern == '' else '{}/{}'.format(pattern,k)
            if isinstance(v,dict):
                _flatten(v,dst,key)
            else:
                dst[key] = v
    result = {}
    _flatten(d,result)
    return result

flatten(ansible_fact) 

{'sda/holders': [],
 'sda/host': 'SATA controller: Intel Corporation 82801HR/HO/HH (ICH8R/DO/DH) 6 port SATA Controller [AHCI mode] (rev 02)',
 'sda/model': 'CentOS Linux-0 S',
 'sda/partitions/sda1/holders': [],
 'sda/partitions/sda1/sectors': '1024000',
 'sda/partitions/sda1/sectorsize': 512,
 'sda/partitions/sda1/size': '500.00 MB',
 'sda/partitions/sda1/start': '2048',
 'sda/partitions/sda1/uuid': '3aec5112-2a1c-4ce0-91de-e9514ba357b8',
 'sda/partitions/sda2/holders': ['VolGroup-lv_root',
  'VolGroup-lv_swap',
  'VolGroup-lv_home'],
 'sda/partitions/sda2/sectors': '133191680',
 'sda/partitions/sda2/sectorsize': 512,
 'sda/partitions/sda2/size': '63.51 GB',
 'sda/partitions/sda2/start': '1026048',
 'sda/partitions/sda2/uuid': 'null'}

来自[运维部落](http://mp.weixin.qq.com/s?__biz=MzIyMDA1MzgyNw==&mid=2651969267&idx=1&sn=d0ee66db1ec08669cf95fc5639a8bdf6&chksm=8c349edfbb4317c9d24aa46dee3ed4f14da138f974ebf1ff7ab14d7598b6baa7e8d40167855e&mpshare=1&scene=1&srcid=0801u75IlL8EzfTDtBKfzxiK#rd![image.png](attachment:image.png))

In [ ]:
ß